In [ ]:
import os
import json

import cv2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import AutoImageProcessor, AutoModelForPreTraining

In [ ]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print("CUDA is available. PyTorch can use the GPU.")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch is using the CPU.")

In [ ]:
# prompt: i want to access drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
num_frames = 16
image_size = 224
FeatureNum = 6

In [ ]:
base_path = "/content/drive/MyDrive/Vision_GYM_Research/Data"
LOG_DIR = "/content/drive/MyDrive/Vision_GYM_Research/tensorboard_logs"

In [ ]:
def load_and_resize_frames(num_video, action, isFront, num_idx, num_frames, size=(224, 224)):
    """
    Loads and resizes video frames from specified file paths based on the action type.

    Args:
        num_video (int): Identifier for the video file.
        action (str): The action being performed ('deadlift', 'squat', 'lunges').
        isFront (int): 1 for frontal frames, 0 for lateral frames.
        num_idx (int): Index for the specific video segment.
        num_frames (int): Number of frames to load.
        size (tuple): Target size for resizing frames (default is (224, 224)).

    Returns:
        list: A list of resized frames.
    """
    frames = []

    action_to_folder = {
        'Deadlift': 'Deadlift_Frames',
        'Squat': 'Squat_Frames',
        'lunges': 'Lunges_Frames'
    }

    if action not in action_to_folder:
        raise ValueError(f"Unknown action: {action}")

    base_folder = os.path.join(base_path, action_to_folder[action])

    # Load and resize frames
    for i in range(1, num_frames + 1):
        path = os.path.join(base_folder, f"{num_video}_idx_{num_idx}_{i}.jpg")
        img = cv2.imread(path)
        if img is not None:
            img_resized = cv2.resize(img, size)
            frames.append(img_resized)
        else:
            print(f"Warning: Could not load image at {path}")
    return frames

In [ ]:
def process_action_data(base_path, action_name):
    """
    Process data for a specific action by loading the corresponding Excel and JSON files,
    adding pose data, and splitting into train, validation, and test sets.

    Args:
        base_path (str): The base directory containing the files.
        action_name (str): The name of the action (e.g., 'squat', 'deadlift', 'lunges').

    Returns:
        tuple: train_df, val_df, test_df DataFrames.
    """
    # Load the Excel file
    excel_file = f"{action_name}_edited.xlsx"
    df = pd.read_excel(os.path.join(base_path, excel_file))

    # Load the JSON files for front and lateral poses
    front_pose_file = f"front_pose_{action_name}.json"
    lat_pose_file = f"lat_pose_{action_name}.json"

    def load_json_as_numpy(json_file):
        with open(json_file, 'r') as file:
            data = json.load(file)
        return np.array(data)

    front_pose_array = load_json_as_numpy(os.path.join(base_path, front_pose_file))
    lat_pose_array = load_json_as_numpy(os.path.join(base_path, lat_pose_file))

    # Ensure `front_pose` and `lat_pose` columns exist
    if 'front_pose' not in df.columns:
        df['front_pose'] = None
    if 'lat_pose' not in df.columns:
        df['lat_pose'] = None

    # Assign the loaded arrays to the DataFrame if lengths match
    if len(front_pose_array) == len(df) and len(lat_pose_array) == len(df):
        df['front_pose'] = list(front_pose_array)
        df['lat_pose'] = list(lat_pose_array)
    else:
        raise ValueError("The length of the loaded arrays does not match the DataFrame.")

    # Shuffle the DataFrame
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Define split ratios
    train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15

    # Calculate the number of samples for each set
    total_samples = len(df)
    train_size = int(total_samples * train_ratio)
    val_size = int(total_samples * val_ratio)

    # Split the DataFrame
    train_df = df.iloc[:train_size]
    val_df = df.iloc[train_size:train_size + val_size]
    test_df = df.iloc[train_size + val_size:]

    return train_df, val_df, test_df

In [ ]:
train_df_squat, val_df_squat, test_df_squat = process_action_data(base_path, 'squat')


# Define the actions
actions = ['squat']

# Process data for each action and store results in dictionaries
splits = {action: process_action_data(base_path, action) for action in actions}

# Concatenate and shuffle DataFrames for each split
train_df = pd.concat([splits[action][0] for action in actions]).sample(frac=1, random_state=1).reset_index(drop=True)
val_df = pd.concat([splits[action][1] for action in actions]).sample(frac=1, random_state=1).reset_index(drop=True)
test_df = pd.concat([splits[action][2] for action in actions]).sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
val_df_squat

,Num Video Frontal,Num Video Lateral,NumIdx,Action,Feet Out 30 F,Score: Frontal( - / 1),Whole Feet Flat On the Floor (1) L,Bend Hips and Knees Simultaniously (1) L,Hips backwards (1) L,Lower back neural (1) L,Hips are lower than knees level (1 point) L,Score: Lateral ( - / 5),Total Score - /6,class,rating,front_pose,lat_pose
206,17,18,4,Squat,1,1,1,0,1,1,0,3,4,1,4,"[[[0.4629118740558624, 0.14725933969020844, -0...","[[[0.4394490718841553, 0.16691318154335022, -0..."
207,93,94,4,Squat,1,1,1,1,1,1,0,4,5,1,5,"[[[0.5139048099517822, 0.2682655155658722, -0....","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
208,65,66,1,Squat,0,0,1,1,1,0,0,3,3,1,3,"[[[0.4789502024650574, 0.41165298223495483, -0...","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
209,17,18,0,Squat,1,1,1,0,1,1,0,3,4,1,4,"[[[0.5205179452896118, 0.09167762845754623, -0...","[[[0.45640236139297485, 0.17358537018299103, -..."
210,51,52,3,Squat,0,0,0,0,0,1,0,1,1,1,1,"[[[0.4467923045158386, 0.24792860448360443, -0...","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
211,105,106,4,Squat,1,1,1,1,1,1,0,4,5,1,5,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
212,63,64,3,Squat,1,1,1,1,1,1,0,4,5,1,5,"[[[0.5316340327262878, 0.20646166801452637, -0...","[[[0.36960580945014954, 0.20044465363025665, -..."
213,9,10,3,Squat,1,1,1,1,1,1,0,4,5,1,5,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...","[[[0.5583255887031555, 0.22064480185508728, -0..."
214,119,120,0,Squat,0,0,1,1,0,1,0,3,3,1,3,"[[[0.5145508050918579, 0.2497655153274536, -0....","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
215,107,108,0,Squat,0,0,1,1,1,1,0,4,4,1,4,"[[[0.553017795085907, 0.2488527148962021, -0.3...","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

class ResNetLSTMModel(nn.Module):
    def __init__(
        self,
        num_frames=16,         # Number of frames in the sequence
        num_classes=6,         # Number of output features
        resnet_hidden_size=512, # Feature size from ResNet
        lstm_hidden_size=256,   # Hidden size of the LSTM
        lstm_num_layers=2,      # Number of LSTM layers
        dropout_rate=0.3        # Dropout rate for regularization
    ):
        super(ResNetLSTMModel, self).__init__()
        self.num_frames = num_frames
        self.lstm_hidden_size = lstm_hidden_size

        # Load pre-trained ResNet-18 for frontal and lateral inputs
        self.resnet_frontal = resnet18(pretrained=True)
        self.resnet_lateral = resnet18(pretrained=True)

        # Modify ResNet to output features instead of classification logits
        self.resnet_frontal.fc = nn.Identity()  # Output raw features
        self.resnet_lateral.fc = nn.Identity()  # Output raw features

        # BatchNorm before feeding to LSTM
        self.bn_frontal = nn.BatchNorm1d(resnet_hidden_size)
        self.bn_lateral = nn.BatchNorm1d(resnet_hidden_size)

        # LSTM for temporal modeling
        self.lstm_frontal = nn.LSTM(
            input_size=resnet_hidden_size,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.lstm_lateral = nn.LSTM(
            input_size=resnet_hidden_size,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_num_layers,
            batch_first=True,
            bidirectional=True
        )

        # Fully connected layers after combining LSTM outputs
        self.fc_combine = nn.Linear(lstm_hidden_size * 2 * 2, 512)
        self.bn_combine = nn.BatchNorm1d(512)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(512, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn_fc2 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, frontal, lateral):
        batch_size, num_frames, channels, height, width = frontal.size()

        # Reshape input for ResNet: [batch_size * num_frames, 3, height, width]
        frontal = frontal.view(batch_size * num_frames, channels, height, width)
        lateral = lateral.view(batch_size * num_frames, channels, height, width)

        # Pass each frame through ResNet
        frontal_features = self.resnet_frontal(frontal)  # [batch_size * num_frames, resnet_hidden_size]
        lateral_features = self.resnet_lateral(lateral)  # [batch_size * num_frames, resnet_hidden_size]

        # Reshape to [batch_size, num_frames, resnet_hidden_size] for LSTM
        frontal_features = frontal_features.view(batch_size, num_frames, -1)
        lateral_features = lateral_features.view(batch_size, num_frames, -1)

        # Apply BatchNorm across temporal features
        frontal_features = self.bn_frontal(frontal_features.transpose(1, 2)).transpose(1, 2)
        lateral_features = self.bn_lateral(lateral_features.transpose(1, 2)).transpose(1, 2)

        # Pass through LSTM
        frontal_lstm_out, _ = self.lstm_frontal(frontal_features)  # [batch_size, num_frames, 2 * lstm_hidden_size]
        lateral_lstm_out, _ = self.lstm_lateral(lateral_features)  # [batch_size, num_frames, 2 * lstm_hidden_size]

        # Take the last hidden state from both LSTMs
        frontal_final = frontal_lstm_out[:, -1, :]  # [batch_size, 2 * lstm_hidden_size]
        lateral_final = lateral_lstm_out[:, -1, :]  # [batch_size, 2 * lstm_hidden_size]

        # Concatenate frontal and lateral features
        combined_features = torch.cat((frontal_final, lateral_final), dim=1)  # [batch_size, 4 * lstm_hidden_size]

        # Pass through fully connected layers with BatchNorm
        x = self.fc_combine(combined_features)
        x = self.bn_combine(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.bn_fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [ ]:
class PoseVideoDataset(Dataset):

    def __init__(self, df, num_frames):
        self.df = df
        self.num_frames = num_frames
        self.train_df_squat = train_df_squat

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Get frontal and lateral video paths and index
        num_video_frontal = row['Num Video Frontal']
        num_video_lateral = row['Num Video Lateral']
        num_idx = row['NumIdx']
        action = row['Action']
        # Extract pose landmarks from video frames
        pose_landmarks_frontal = row['front_pose']
        pose_landmarks_lateral = row['lat_pose']

        video_frames_frontal = load_and_resize_frames(num_video_frontal, action, 1, num_idx, self.num_frames)
        video_frames_lateral = load_and_resize_frames(num_video_lateral, action, 0, num_idx, self.num_frames)
        # pose_landmarks_tensor_frontal = torch.cat((pose_landmarks_tensor_frontal, distances_frontal), dim=0)
        # pose_landmarks_tensor_lateral = torch.cat((pose_landmarks_tensor_lateral, distances_lateral), dim=0)
        # Get labels (action class)

        # Convert video frames to tensors and normalize
        video_frames_frontal = torch.tensor(video_frames_frontal, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
        video_frames_lateral = torch.tensor(video_frames_lateral, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
        label_class = torch.tensor(row['class'], dtype=torch.long)

        ratings = self._process_ratings(self.train_df_squat,row)

        # Convert ratings to tensor
        ratings = torch.tensor(ratings, dtype=torch.float32) if ratings is not None else None

        return (video_frames_frontal, video_frames_lateral, label_class, ratings)

    def _process_ratings(self, df,row):
        """
        Process the ratings DataFrame to extract and normalize scores.

        Args:
            df (pd.DataFrame): DataFrame containing ratings for the specific action.

        Returns:
            list: Normalized and thresholded scores.
        """
        # Select columns ending with 'F' or 'L'
        relevant_columns = [col for col in df.columns if col.endswith('F') or col.endswith('L')]

        # Extract ratings and normalize
        scores = row[relevant_columns].values


        # Apply threshold: Convert to 0 or 1 based on specific thresholding conditions
        thresholded_scores = np.where(scores >= 0.5, 1, 0)

        return thresholded_scores.tolist()  # Return mean of the scores for each sample

In [ ]:
train_df['Num Video Frontal'][0]

5

In [ ]:
def create_dataloader(df, num_frames, batch_size=16, shuffle=True):
    """
    Creates a PyTorch DataLoader from the PoseVideoDataset.

    Args:
        df (pd.DataFrame): DataFrame containing the dataset information.
        num_frames (int): The number of frames to extract from each video.
        batch_size (int, optional): Batch size for the DataLoader. Defaults to 8.
        shuffle (bool, optional): Whether to shuffle the data. Defaults to True.

    Returns:
        DataLoader: A PyTorch DataLoader for the dataset.
    """
    dataset = PoseVideoDataset(df, num_frames)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [ ]:
def train_combined_model(CustomSwinTransformerModel, train_dataloader, eval_dataloader, epochs=1000, lr=1e-4,
                         device='cpu', clip_grad_norm=1.0, patience=10):
    # TensorBoard setup
    writer = SummaryWriter(log_dir=LOG_DIR)

    CustomSwinTransformerModel.to(device)

    # Set up optimizer and loss functions
    optimizer = optim.Adam(list(CustomSwinTransformerModel.parameters()), lr=lr)
    feature_loss = nn.BCEWithLogitsLoss()  # Loss for rating task (binary)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)
    best_eval_loss = float('inf')
    patience_counter = 0
    print("Training the model...")
    a = 0
    for epoch in range(epochs):
        CustomSwinTransformerModel.train()
        running_loss = 0.0
        print(f'Start of Epoch {epoch+1}')
        for batch_idx, batch in enumerate(train_dataloader):
            # Calculate progress percentage
            progress = (batch_idx + 1) / len(train_dataloader) * 100
            # Unpack batch data and move to the specified device
            (num_video_frontal,num_video_lateral,label_class, ratings) = [tensor.to(device) for tensor in batch]
            optimizer.zero_grad()
            # Forward pass through the classification model
            ratings_output =CustomSwinTransformerModel(num_video_frontal,num_video_lateral).to(device)

            loss = feature_loss(ratings_output, ratings)


            # Backward pass and optimization
            loss.backward()
            torch.nn.utils.clip_grad_norm_(list(CustomSwinTransformerModel.parameters()), clip_grad_norm)
            optimizer.step()

            running_loss += loss.item()

            # Print progress during each epoch
            print(f"Epoch [{epoch + 1}/{epochs}], Progress: {progress:.2f}%, Batch [{batch_idx + 1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

        avg_loss = running_loss / len(train_dataloader)

        # Log training loss to TensorBoard
        writer.add_scalar("Loss/Train", avg_loss, epoch)

        # Evaluate both models after each epoch
        eval_loss, hamming_distances, metrics = evaluate_combined_model(CustomSwinTransformerModel, eval_dataloader, feature_loss, device)
        mean_hamming_distance = sum(hamming_distances.values()) / len(hamming_distances) if len(hamming_distances) > 0 else 0.0


        tp = metrics['TP']
        tn = metrics['TN']
        fp = metrics['FP']
        fn = metrics['FN']

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        jaccard_index = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0.0

        # Log validation metrics to TensorBoard
        writer.add_scalar("Loss/Validation", eval_loss, epoch)
        writer.add_scalar("Loss/Hamming Loss", mean_hamming_distance, epoch)
        writer.add_scalar(f"Metrics/Precision", precision, epoch)
        writer.add_scalar(f"Metrics/Recall", recall, epoch)
        writer.add_scalar(f"Metrics/F1-Score", f1_score, epoch)
        writer.add_scalar(f"Metrics/jaccard index", jaccard_index, epoch)

        # Print summary for each epoch
        print(f"Epoch [{epoch + 1}/{epochs}] Summary: "
              f"Train Loss: {avg_loss:.4f}, Eval Loss: {eval_loss:.4f}",
              f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}")
        print(f"Hamming Loss: {mean_hamming_distance:.4f}, jaccard index: {jaccard_index:.4f}", )

        scheduler.step(eval_loss)

        # Early stopping and model saving
        if eval_loss < best_eval_loss:
            best_eval_loss = eval_loss
            patience_counter = 0
            torch.save(CustomSwinTransformerModel.state_dict(), f"best_rating_model_epoch_{epoch + 1}.pt")
            print("Model checkpoint saved.")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    writer.close()  # Close TensorBoard writer
    print("Training complete.")


In [ ]:
def evaluate_combined_model(CustomSwinTransformerModel, dataloader, feature_loss, device):
    """
    Evaluates the model and computes evaluation metrics for each feature.

    Args:
        CustomSwinTransformerModel (torch.nn.Module): The rating prediction model.
        dataloader (DataLoader): A DataLoader providing the evaluation data.
        feature_loss (nn.Module): The loss function for ratings.
        device (str): The device to perform evaluation on ('cpu' or 'cuda').

    Returns:
        float: Average evaluation loss.
        dict: Hamming distance for squat features.
        dict: TP, TN, FP, FN for each feature.
    """
    CustomSwinTransformerModel.eval()
    total_loss = 0.0
    num_features = FeatureNum
    metrics = {
            feature_idx: {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0} for feature_idx in range(num_features)
        }
    total_samples = {feature_idx: 0 for feature_idx in range(num_features)}

    with torch.no_grad():
        a=0
        for batch_idx, batch in enumerate(dataloader):
            # Move batch data to the device
            print(f"a ={a},  batch_idx = {batch_idx}")
            a=a+1
            num_video_frontal, num_video_lateral, label_class, ratings = [tensor.to(device) for tensor in batch]

            # Predict ratings using the model
            ratings_output = CustomSwinTransformerModel(num_video_frontal, num_video_lateral).to(device)
            total_loss += feature_loss(ratings_output, ratings)

            # Sigmoid activation for binary predictions
            predicted_ratings = torch.sigmoid(ratings_output) > 0.5
            actual_ratings = ratings.byte()

            # Compute TP, TN, FP, FN for each feature
            for feature_idx in range(num_features):
                preds = predicted_ratings[:, feature_idx]
                trues = actual_ratings[:, feature_idx]

                metrics[feature_idx]['TP'] += (preds & trues).sum().item()
                metrics[feature_idx]['TN'] += (~preds & ~trues).sum().item()
                metrics[feature_idx]['FP'] += (preds & ~trues).sum().item()
                metrics[feature_idx]['FN'] += (~preds & trues).sum().item()
                total_samples[feature_idx] += trues.numel()

    # Calculate Hamming distance for each feature
    hamming_distances = {}
    for feature_idx in range(num_features):
          hamming_distances[feature_idx] = (
              metrics[feature_idx]['FP'] + metrics[feature_idx]['FN']
          ) / total_samples[feature_idx] if total_samples[feature_idx] > 0 else 0.0

    avg_loss = total_loss / len(dataloader)

    total_metrics = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}
    # Print TP, TN, FP, FN for each feature
    for feature_idx in range(num_features):
        print(f"Feature {feature_idx}:")
        print(f"  TP: {metrics[feature_idx]['TP']}")
        print(f"  TN: {metrics[feature_idx]['TN']}")
        print(f"  FP: {metrics[feature_idx]['FP']}")
        print(f"  FN: {metrics[feature_idx]['FN']}")
        print(f"  Hamming Loss: {hamming_distances[feature_idx]:.4f}")
        total_metrics['TP'] += metrics[feature_idx]['TP']
        total_metrics['TN'] += metrics[feature_idx]['TN']
        total_metrics['FP'] += metrics[feature_idx]['FP']
        total_metrics['FN'] += metrics[feature_idx]['FN']

    return avg_loss, hamming_distances, total_metrics


In [ ]:
dataloader = create_dataloader(train_df_squat, 16, batch_size=32)
eval_dataloader = create_dataloader(val_df_squat, 16, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(len(dataloader.dataset))  # Check dataset length
print(dataloader.dataset[0][0].shape)  # Try accessing the first element


206


<ipython-input-11-2aab4372a881>:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  video_frames_frontal = torch.tensor(video_frames_frontal, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0


torch.Size([16, 3, 224, 224])


In [ ]:
# Initialize the model
rating_model = CustomSwinTransformerModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

# Train the model
train_combined_model(rating_model, dataloader, eval_dataloader, epochs=1000, lr=1e-4, device=device)

cuda
Training the model...
Start of Epoch 1
Epoch [1/1000], Progress: 14.29%, Batch [1/7], Loss: 0.6991
Epoch [1/1000], Progress: 28.57%, Batch [2/7], Loss: 0.6877
Epoch [1/1000], Progress: 42.86%, Batch [3/7], Loss: 0.6768
Epoch [1/1000], Progress: 57.14%, Batch [4/7], Loss: 0.6644
Epoch [1/1000], Progress: 71.43%, Batch [5/7], Loss: 0.6699
Epoch [1/1000], Progress: 85.71%, Batch [6/7], Loss: 0.6677
Epoch [1/1000], Progress: 100.00%, Batch [7/7], Loss: 0.6351
a =0,  batch_idx = 0
a =1,  batch_idx = 1
Feature 0:
  TP: 25
  TN: 0
  FP: 19
  FN: 0
  Hamming Loss: 0.4318
Feature 1:
  TP: 34
  TN: 0
  FP: 10
  FN: 0
  Hamming Loss: 0.2273
Feature 2:
  TP: 34
  TN: 0
  FP: 10
  FN: 0
  Hamming Loss: 0.2273
Feature 3:
  TP: 0
  TN: 19
  FP: 0
  FN: 25
  Hamming Loss: 0.5682
Feature 4:
  TP: 43
  TN: 0
  FP: 1
  FN: 0
  Hamming Loss: 0.0227
Feature 5:
  TP: 0
  TN: 38
  FP: 0
  FN: 6
  Hamming Loss: 0.1364
Epoch [1/1000] Summary: Train Loss: 0.6715, Eval Loss: 0.6329 Precision: 0.7727, Recall

In [ ]:
train_combined_model(rating_model, dataloader, eval_dataloader, epochs=1000, lr=1e-4, device=device)

Training the model...
Start of Epoch 1
3.8461538461538463
Epoch [1/1000], Progress: 3.85%, Batch [1/26], Loss: 0.5717


KeyboardInterrupt: 